# ME Bank Policy Portal Sraper
Suse a simple login.

FOr now loggign in with the browser, then cut and past the auth cookies into this notebook.

Scraper Code
1. Only searchs for links in dic.container
2. does niot use HTML extention
   

In [1]:
from  scraper import WebScraper
from loguru import logger
import os

class MEB(WebScraper):
    def __init__(self, domain:str, cookie:str, download_path:str, search_only:bool=False):
        super().__init__(domain, cookie, download_path, class_names = None)
        self.search_only = search_only

    def scrape(self, url:str, depth:int, starting_page:bool = True, ) -> None:

        if url[-1] == "/":
            url = url[:-1]
            
        logger.info(f"Scraping {depth}: {url}")
        if self.skip(url, depth, ['/~/', '/meet-the-team/']):
            return
        
        _, extension = self.get_path(url)
        assert extension in [".html"], f"Invalid extension '{extension}' for {url}"

        # download unless except starting page
        html_content = self.save_url(url, starting_page)
        
        # extract all the links and recurse
        self.extract(html_content, depth, "div.right-column" ) # "div." or "div.rn_Container" or div#content
        
        return

# Download MEB Portal

In [ ]:
import shutil

directory_path = r"/home/kevinmcisaac/Projects/policy-pal-pages/policy/MEB/Portal"

if os.path.exists(directory_path):
    shutil.rmtree(directory_path)
os.makedirs(directory_path)


meb = MEB(
    "https://mebroker.powerappsportals.com",
    cookie="Dynamics365PortalAnalytics=GJ6iKY1wcHsYsU1p6GaEaLKQZQn4aFHSeDjMTb6QicYcg2YZr57rmayej_pO4DMPHrK16OZzzZtclEjb7xmJCRmLlz0t-sqW8SWSgJPnitiP1YkV-sR0xllVw1OxNFPpyipcB3jDqQ9iBjMmpETJag2; __RequestVerificationToken=1vyAipgUu2bGT0qTzi6gdrmmPKUVArfEyajhgViKj0_lwvweNQjlNACF-nJzwNZ-5NeB0fsQOTVVTGLzZfC3JoNAdNc7hpiKXwX-Kyx8Cyg1; ARRAffinity=ae5dafc3502301c5972f13d1cd832204eb53fc43da4059b5dcd299d08fa97ae7; ARRAffinitySameSite=ae5dafc3502301c5972f13d1cd832204eb53fc43da4059b5dcd299d08fa97ae7; timezoneoffset=-480; isDSTSupport=false; isDSTObserved=false; ContextLanguageCode=en-US; ai_user=Qd/Bahtpkw206vdATb+jXu|2025-03-05T22:57:45.640Z; timeZoneCode=210; .AspNet.ApplicationCookie=nU_SwbsFUAPBRRIGdJdo1mhn09AbeM15esVVLmTC4NVrxCTe2BhrzWimlL38jE3VOhgi64Ud-5sw8JDJkopPqx2w3IfMu5YAPiOpQMonNvVLubefpULzhJNSDFBcRT0dvRjaqYswppjsMNL_v7JJPBSWNV-qTRR6Qb88M7P3_8X_nfhuBAny49zbOsIruUfgCqad5zW9V4S2H_H6zzPv8KuT74z24t93NpxG-hVFCD6DMNf_znWC3_r_PKoCgbt0uv-gIvwMh0EPpkJ7kjNe_t8Rn28FIl3VYqwm4Mm5ClJ-GLDbufpRXHwtvhuLYcJqCsVX98mgNzl2K9WcCOZVFNlKCqVOsAtUyI3Am9l-iEjoMg1bmWaKcfeTdlGEDXJA5eFa3V4OaOzQ7UB5Dwr3ZvIIu-4XqfSfQTwgn-ossH3Jjk5BqmeeC-rBnJupzP-UWwrfwVbbFZZXGGE9yQ_0UB8SfVRjJANVyrzOOs1YfprDMTdvdcYbFfZTmfz523pOwQRDh4EvfdNm_H9FTWddt0BpHsqO7IoXKazC2vCD3lbz0Elr7vrHQX9nNMzDP-EFZuXv_ZOIRoh5LZaugmqaeQ; ai_session=38nnUZHZBqomTdmfjHC671|1741224081442|1741225462812",
    download_path=directory_path,
    search_only=False,
)
# 6 got 75, 12 got 83
meb.scrape(
    "https://mebroker.powerappsportals.com/policy-home1/home-lending-policy/", depth=12
)


print(len(meb.download))

2025-03-06 10:09:15.298 | INFO     | __main__:scrape:15 - Scraping 12: https://mebroker.powerappsportals.com/policy-home1/home-lending-policy


2025-03-06 10:09:16.543 | INFO     | __main__:scrape:15 - Scraping 11: https://mebroker.powerappsportals.com/policy-home1/home-lending-policy/about-this-policy
2025-03-06 10:09:17.386 | INFO     | scraper:save_content:56 - Downloading https://mebroker.powerappsportals.com/policy-home1/home-lending-policy/about-this-policy to /home/kevinmcisaac/Projects/policy-pal-pages/policy/MEB/Portal/policy-home1/home-lending-policy/about-this-policy.html
2025-03-06 10:09:17.440 | INFO     | __main__:scrape:15 - Scraping 10: https://mebroker.powerappsportals.com/policy-home1/home-lending-policy/about-this-policy/definition
2025-03-06 10:09:18.281 | INFO     | scraper:save_content:56 - Downloading https://mebroker.powerappsportals.com/policy-home1/home-lending-policy/about-this-policy/definition to /home/kevinmcisaac/Projects/policy-pal-pages/policy/MEB/Portal/policy-home1/home-lending-policy/about-this-policy/definition.html
2025-03-06 10:09:18.328 | INFO     | __main__:scrape:15 - Scraping 9: https

83
